In [1]:
import numpy as np
import pandas as pd
import io
from google.colab import files
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import model_selection
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold, train_test_split, cross_val_score, KFold, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression,Perceptron,RidgeClassifier,RidgeClassifierCV,SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier,VotingClassifier,\
StackingClassifier,VotingClassifier,HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import time
from scipy.stats import skew
from scipy import stats
from numpy import polyfit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, f1_score
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline

from collections import Counter
from numpy import mean
from sklearn.datasets import make_classification

import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.8 MB/s eta 0:00:00


filter function
create a list of unique players
list of matches

Last_Player_rating	= filter by player name then rolling 1 and shift 1
Avg_player_rating		= filter by player name then rolling 5 and shift 1
Squad_Last_rating	 = loop  whole db (loop1) then funct filter by Match_id by current Match_id and team name by current Match_id then rolling 1 and shift-1

Squad_Avg_att_rating	= loop  whole db (loop1) then funct filter by Match_id by current Match_id, team name in current Match_id and main_pos = att
get avg
then rolling 1 and shift-1

Squad_Avg_mid_rating
Squad_Avg_def_rating
Squad_Avg_gk_rating
Squad_Avg_GF
Squad_avg_GA


In [2]:
import numpy as np
import pandas as pd
import io
from google.colab import files
import codecs

# Load the dataset 2023 data
uploaded = files.upload()



Saving 2022_2023 fixed_player_data.xlsx to 2022_2023 fixed_player_data.xlsx


In [3]:
# data = pd.read_csv('2023_data (2).csv', encoding='unicode_escape')
data = pd.read_excel('2022_2023 fixed_player_data.xlsx', sheet_name='raw')
# matches = pd.read_excel('2023_matches.csv', encoding='unicode_escape')
# matches_bk = matches.copy()
data_bk =  data.copy()

In [4]:
# Calculate last player rating, average player rating, and previous game add weights based on position
data_bk['Last_Player_rating'] = data_bk.groupby('player')['Player_Rating'].shift(1)
data_bk['Avg_player_rating'] = data_bk.groupby('player')['Player_Rating'].rolling(3).mean().shift(1).reset_index(level=0, drop=True)
data_bk['Previous_Game'] = data_bk.groupby('player')['Match_id'].shift(1)
data_bk['avg_total'] = data_bk.groupby('player')['total'].rolling(3).mean().shift(1).reset_index(level=0, drop=True)
data_bk['avg_shotot'] = data_bk.groupby('player')['shotot'].rolling(3).mean().shift(1).reset_index(level=0, drop=True)
data_bk['avg_KeyPassTotal'] = data_bk.groupby('player')['KeyPassTotal'].rolling(3).mean().shift(1).reset_index(level=0, drop=True)
data_bk['avg_PassSuccessInMatch'] = data_bk.groupby('player')['PassSuccessInMatch'].rolling(3).mean().shift(1).reset_index(level=0, drop=True)
data_bk['avg_duelAerialWon'] = data_bk.groupby('player')['duelAerialWon'].rolling(3).mean().shift(1).reset_index(level=0, drop=True)
data_bk['avg_Touches'] = data_bk.groupby('player')['Touches'].rolling(3).mean().shift(1).reset_index(level=0, drop=True)

# Iterate through the DataFrame avg_Touches,avg_duelAerialWon,avg_PassSuccessInMatch,avg_KeyPassTotal,avg_shotot,avg_total
for index, row in data_bk.iterrows():
    if pd.isnull(row['Previous_Game']):
        match_id = row['Match_id']
        players_team = row['players_team']
        opponent_team = row['opponent_team']

        previous_game_candidate = data_bk[
            (data_bk['Match_id'] == match_id) &
            (data_bk['players_team'] == players_team) &
            (data_bk['opponent_team'] == opponent_team)
        ]['Previous_Game'].values[0]

        if pd.notnull(previous_game_candidate):
            data_bk.at[index, 'Previous_Game'] = previous_game_candidate
        else:
            data_bk.at[index, 'Previous_Game'] = match_id

# Calculate squad average rating for all players with the same 'players_team' and 'Previous_Game'
data_bk['Squad_Last_rating'] = data_bk.groupby(['players_team', 'Previous_Game'])['Player_Rating'].transform('mean')

# Calculate squad average rating for all players with the same 'players_team' and 'Previous_Game' regardless of 'main_Pos'
data_bk['Squad_Avg_att_rating'] = data_bk.groupby(['players_team', 'Previous_Game'])['Player_Rating'].transform(lambda x: x[data_bk['main_Pos'].str.contains("Att")].mean())
data_bk['Squad_Avg_mid_rating'] = data_bk.groupby(['players_team', 'Previous_Game'])['Player_Rating'].transform(lambda x: x[data_bk['main_Pos'].str.contains("Mid")].mean())
data_bk['Squad_Avg_def_rating'] = data_bk.groupby(['players_team', 'Previous_Game'])['Player_Rating'].transform(lambda x: x[data_bk['main_Pos'].str.contains("Def")].mean())
data_bk['Squad_Avg_gk_rating'] = data_bk.groupby(['players_team', 'Previous_Game'])['Player_Rating'].transform(lambda x: x[data_bk['main_Pos'].str.contains("GK")].mean())



In [5]:
# Latest Version
# Create a dictionary to store opponent team's last match IDs and ratings
opponent_last_match_ratings = {}

# Iterate through the rows of the DataFrame
for index, main_row in data_bk.iterrows():
    var_opponent_team = str(main_row['opponent_team'])
    var_Player_MatchID = str(main_row['Match_id'])
    var_Player_TeamName = str(main_row['players_team'])
    var_Player_Name = str(main_row['player'])
    var_MatchId_PlayerTeam = var_Player_MatchID + var_Player_TeamName
    print(index, end='\r')

    # Check if the opponent's last match ID and ratings are already stored
    if (var_opponent_team, var_Player_MatchID) in opponent_last_match_ratings:
        opponent_data = opponent_last_match_ratings[(var_opponent_team, var_Player_MatchID)]
        last_match_id = opponent_data['Opp_Last_Match']
        var_squad_avg = opponent_data['rating']
        var_squad_avg_Att = opponent_data['att_rating']
        var_squad_avg_Mid = opponent_data['mid_rating']
        var_squad_avg_Def = opponent_data['def_rating']
        var_squad_avg_GK = opponent_data['GK_rating']
    else:
        # If not stored, find the previous game's Match_id and ratings for the opponent
        filtered_df = data_bk[data_bk['players_team'] == var_opponent_team]
        unique_match_ids = filtered_df['Match_id'].drop_duplicates().reset_index(drop=True)

        # Select the fixture after the current fixture in the opp fixture list to get the last game they played
        opp_index = unique_match_ids[unique_match_ids == var_Player_MatchID].index[0]
        previous_game_index = opp_index - 1
        if previous_game_index >= 0:
            last_match_id = unique_match_ids[previous_game_index]
            # Select AVG rating for a team
            get_Previous_Opp_Game_PlayerRatings = data_bk[(data_bk['Match_id'] == last_match_id) & (data_bk['players_team'] == var_opponent_team)]
            var_squad_avg = get_Previous_Opp_Game_PlayerRatings['Player_Rating'].mean()
            # Select data for Att
            get_Opp_Data_Att = data_bk[(data_bk['Match_id'] == last_match_id) & (data_bk['players_team'] == var_opponent_team) & (data_bk['main_Pos'] == "Att")]
            var_squad_avg_Att = get_Opp_Data_Att['Player_Rating'].mean()
            # Select data for Mid
            get_Opp_Data_Mid = data_bk[(data_bk['Match_id'] == last_match_id) & (data_bk['players_team'] == var_opponent_team) & (data_bk['main_Pos'] == "Mid")]
            var_squad_avg_Mid = get_Opp_Data_Mid['Player_Rating'].mean()
            # Select data for Def
            get_Opp_Data_Def = data_bk[(data_bk['Match_id'] == last_match_id) & (data_bk['players_team'] == var_opponent_team) & (data_bk['main_Pos'] == "Def")]
            var_squad_avg_Def = get_Opp_Data_Def['Player_Rating'].mean()
            # Select data for GK
            get_Opp_Data_GK = data_bk[(data_bk['Match_id'] == last_match_id) & (data_bk['players_team'] == var_opponent_team) & (data_bk['main_Pos'] == "GK")]
            var_squad_avg_GK = get_Opp_Data_GK['Player_Rating'].mean()

        else:
            #if the opp previous match doesnt exisit use current match as previous match and 6.0 for ratings
            last_match_id = var_MatchId_PlayerTeam
            var_squad_avg = 6.0
            var_squad_avg_Att = 6.0
            var_squad_avg_Mid = 6.0
            var_squad_avg_Def = 6.0
            var_squad_avg_GK = 6.0

        # Store the opponent's last match ID and ratings in the dictionary
        opponent_last_match_ratings[(var_opponent_team, var_Player_MatchID)] = {
            'Player': var_Player_Name,
            'Player_team': var_Player_TeamName,
            'OPP_team': var_opponent_team,
            'Match_id': var_MatchId_PlayerTeam,
            'Opp_Last_Match': last_match_id,
            'rating': var_squad_avg,
            'att_rating': var_squad_avg_Att,
            'mid_rating': var_squad_avg_Mid,
            'def_rating': var_squad_avg_Def,
            'GK_rating': var_squad_avg_GK,
        }

    # Update the main DataFrame with the opponent's last match ratings
    data_bk.at[index, 'Opp_Last_Team_rating'] = var_squad_avg
    data_bk.at[index, 'Opp_Previous_Game'] = last_match_id
    data_bk.at[index, 'Opp_avg_att_rating'] = var_squad_avg_Att
    data_bk.at[index, 'Opp_avg_mid_rating'] = var_squad_avg_Mid
    data_bk.at[index, 'Opp_avg_def_rating'] = var_squad_avg_Def
    data_bk.at[index, 'Opp_avg_gk_rating'] = var_squad_avg_GK
    # print(var_Player_Name,var_Player_TeamName,var_opponent_team, var_MatchId_PlayerTeam, last_match_id,var_squad_avg, var_squad_avg_Att)
    # print(var_Player_Name,var_Player_TeamName,var_opponent_team, var_MatchId_PlayerTeam, last_match_id,var_squad_avg, var_squad_avg_Att)



In [6]:
# handling missing data
data_bk['Avg_player_rating'] = 0.0
data_bk['Last_Player_rating'] = 0.0

for index, row in data_bk.iterrows():
    player = row['player']
    prev_ratings = data_bk[(data_bk['player'] == player) & (data_bk.index < index)]['Player_Rating']

    if len(prev_ratings) >= 1:
        last_rating = prev_ratings.iloc[-1]
    else:
        last_rating = row['Player_Rating']

    data_bk.at[index, 'Last_Player_rating'] = last_rating

for index, row in data_bk.iterrows():
    player = row['player']
    prev_ratings = data_bk[(data_bk['player'] == player) & (data_bk.index < index)]['Last_Player_rating']

    if len(prev_ratings) >= 3:
        avg_rating = prev_ratings[-3:].mean()
    elif len(prev_ratings) == 2:
        avg_rating = prev_ratings.mean()
    elif len(prev_ratings) == 1:
        avg_rating = prev_ratings.iloc[0]
    else:
        avg_rating = row['Last_Player_rating']

    data_bk.at[index, 'Avg_player_rating'] = avg_rating



In [7]:
# handling missing data
cols_to_average_to = ['avg_Touches', 'avg_duelAerialWon', 'avg_PassSuccessInMatch', 'avg_KeyPassTotal', 'avg_shotot', 'avg_total']

cols_to_average_from = ['Touches', 'duelAerialWon', 'PassSuccessInMatch', 'KeyPassTotal', 'shotot', 'total']

for col_from, col_to in zip(cols_to_average_from, cols_to_average_to):
    for index, row in data_bk.iterrows():
        player = row['player']
        prev_values = data_bk[(data_bk['player'] == player) & (data_bk.index < index)][col_from]

        if len(prev_values) >= 3:
            avg_value = prev_values[-3:].mean()
        elif len(prev_values) == 2:
            avg_value = prev_values.mean()
        elif len(prev_values) == 1:
            avg_value = prev_values.iloc[0]
        else:
            avg_value = row[col_from]

        data_bk.at[index, col_to] = avg_value


In [ ]:

# Fill missing cols
for index, row in data_bk.iterrows():
    match_id = row['Match_id']
    players_team = row['players_team']
    opponent_team = row['opponent_team']

    if pd.isnull(row['Squad_Avg_att_rating']):
        squad_avg_att_candidate = data_bk[
            (data_bk['Match_id'] == match_id) &
            (data_bk['players_team'] == players_team) &
            (data_bk['opponent_team'] == opponent_team)
        ]['Squad_Avg_att_rating'].dropna().iloc[0]
        data_bk.at[index, 'Squad_Avg_att_rating'] = squad_avg_att_candidate

    if pd.isnull(row['Squad_Avg_mid_rating']):
        squad_avg_mid_candidate = data_bk[
            (data_bk['Match_id'] == match_id) &
            (data_bk['players_team'] == players_team) &
            (data_bk['opponent_team'] == opponent_team)
        ]['Squad_Avg_mid_rating'].dropna().iloc[0]
        data_bk.at[index, 'Squad_Avg_mid_rating'] = squad_avg_mid_candidate

    if pd.isnull(row['Squad_Avg_def_rating']):
        squad_avg_def_candidate = data_bk[
            (data_bk['Match_id'] == match_id) &
            (data_bk['players_team'] == players_team) &
            (data_bk['opponent_team'] == opponent_team)
        ]['Squad_Avg_def_rating'].dropna().iloc[0]
        data_bk.at[index, 'Squad_Avg_def_rating'] = squad_avg_def_candidate

    if pd.isnull(row['Squad_Avg_gk_rating']):
        squad_avg_gk_candidate = data_bk[
            (data_bk['Match_id'] == match_id) &
            (data_bk['players_team'] == players_team) &
            (data_bk['opponent_team'] == opponent_team)
        ]['Squad_Avg_gk_rating'].dropna().iloc[0]
        data_bk.at[index, 'Squad_Avg_gk_rating'] = squad_avg_gk_candidate



In [ ]:
# Define the rating ranges and corresponding labels
rating_ranges = [(0, 3.9), (4.0, 4.9), (5.0, 5.9), (6.0, 6.9), (7.0, 7.9), (8.0, 8.9), (9.0, 10)]
labels = ['Extremely Poor', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good', 'Excellent']

# Create a new column 'Player_Perf' using pandas' cut function
data_bk['Player_Perf'] = pd.cut(data_bk['Player_Rating'],
                                 bins=[r[0] for r in rating_ranges] + [rating_ranges[-1][1] + 1],
                                 labels=labels,
                                 right=False)

# Mapping venue to eiher home or away
stadium_team_mapping = {
    'Selhurst Park': 'Crystal Palace',
    'Tottenham Hotspur Stadium': 'Tottenham',
    'Elland Road': 'Leeds',
    'Goodison Park': 'Everton',
    'Old Trafford': 'Man Utd',
    'Craven Cottage': 'Fulham',
    'London Stadium': 'West Ham',
    'St. James\' Park': 'Newcastle',
    'Vitality Stadium': 'Bournemouth',
    'King Power Stadium': 'Leicester',
    'Molineux Stadium': 'Wolves',
    'Gtech Community Stadium': 'Brentford',
    'The City Ground': 'Nottingham Forest',
    'Stamford Bridge': 'Chelsea',
    'St. Mary\'s Stadium': 'Southampton',
    'Amex Stadium': 'Brighton',
    'Emirates Stadium': 'Arsenal',
    'Villa Park': 'Aston Villa',
    'Etihad Stadium': 'Man City',
    'Anfield': 'Liverpool'
}

data_bk['venue'] = data_bk.apply(lambda row: 'Home' if stadium_team_mapping.get(row['stadium']) == row['players_team'] else 'Away', axis=1)



In [10]:
# check the distribution of the player rating
import plotly.express as px
df = data_bk["Player_Perf"].value_counts().sort_values()
fig = px.bar(df, x='Player_Perf',
             hover_data=['Player_Perf'], color='Player_Perf',text_auto=True,
             labels={'pop':'distrubution of Player_Perf'}, height=400, width=400)

fig.show()

In [ ]:
# Calculatng opponents strenght based on previous season
# Create a mapping of top 10 teams
top_10_teams = ['Man City', 'Liverpool', 'Chelsea', 'Tottenham', 'Arsenal', 'Man Utd', 'West Ham', 'Leicester', 'Brighton']

# Function to assign 'top_10' or 'bottom_10' based on team name
def assign_team_power(team):
    if team in top_10_teams:
        return 'top_10'
    else:
        return 'bottom_10'

# Apply the function to the 'opponent_team' column
data_bk['opponent_team_pwr'] = data_bk['opponent_team'].apply(assign_team_power)


In [12]:
# Calculate the quartiles of player performance

# Define the labels for each tertile
# tertile_labels = ['Good', 'Average', 'Poor']

# Calculate tertiles for 'Player_Rating' and store in 'Player_Perf' column
# data_bk['Player_Perf'] = pd.qcut(data_bk['Player_Rating'], q=3, labels=tertile_labels)

# Calculate tertiles for 'Player_Rating' and store in 'Player_Perf' column
# data_bk['attendance'] = pd.qcut(data_bk['attendance'], q=3, labels=tertile_labels)

# Calculate tertiles for 'Player_Rating' and store in 'Player_Perf' column
# data_bk['Opp_Avg_team_rating'] = pd.qcut(data_bk['Opp_Last_Team_rating'], q=3, labels=tertile_labels)



# Calculate terciles for 'Player_Rating' column
terciles = data_bk['Player_Rating'].quantile([0.33, 0.66])

# Calculate terciles for 'attendance' column
terciles_attendance = data_bk['attendance'].quantile([0.33, 0.66])

# Calculate terciles for 'Opp_Avg_team_rating' column
terciles_team_rating = data_bk['Opp_Avg_team_rating'].quantile([0.33, 0.66])

# Define the function to assign labels based on terciles
def classify_performance(rating):
    if rating > terciles[0.66]:
        return 'Good'
    elif rating > terciles[0.33]:
        return 'Average'
    else:
        return 'Poor'

# Define the function to assign labels based on terciles
def classify_attendance(attendance):
    if attendance > terciles_attendance[0.66]:
        return 'High'
    elif attendance > terciles_attendance[0.33]:
        return 'Moderate'
    else:
        return 'Low'

def classify_team_rating(rating):
    if rating > terciles_team_rating[0.66]:
        return 'Strong'
    elif rating > terciles_team_rating[0.33]:
        return 'Moderate'
    else:
        return 'Weak'

# Apply the function to create the 'Player_Perf' column
data_bk['Player_Perf'] = data_bk['Player_Rating'].apply(classify_performance)

# Apply the functions to create the new columns
data_bk['attendance'] = data_bk['attendance'].apply(classify_attendance)
data_bk['Opp_Avg_team_rating'] = data_bk['Opp_Last_Team_rating'].apply(classify_team_rating)

In [13]:
# Calculate the classfication of player performance

# Calculate the average player rating
average_player_rating = data_bk['Player_Rating'].mean()

# Set Player_Perf based on player rating compared to the average
data_bk['New_Player_Perf'] = data_bk['Player_Rating'].apply(lambda x: 'Above Average' if x > average_player_rating else 'Below Average')



In [250]:
# export adjusted data
data_bk.to_excel('test_arranged.xlsx', index=False)
files.download('test_arranged.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# check for missing data
nan_zero_attendance_rows = data_bk[(data_bk['attendance'].isnull()) | (data_bk['attendance'] == 0)]

nan_zero_attendance_rows

,newer_id,new_ID,ID,player,Match Week,match_date,season,match_day,players_team,opponent_team,...,avg_total,avg_shotot,avg_KeyPassTotal,avg_PassSuccessInMatch,avg_duelAerialWon,avg_Touches,Opp_Previous_Game,Player_Perf,venue,opponent_team_pwr


In [ ]:
# check the distribution of the player rating classification
import plotly.express as px
df = data_bk["Player_Perf"].value_counts().sort_values()
fig = px.bar(df, x='Player_Perf',
             hover_data=['Player_Perf'], color='Player_Perf',text_auto=True,
             labels={'pop':'distrubution of Player_Perf'}, height=400, width=400)

fig.show()

In [ ]:
# check the distribution of the player rating class based on above or below average
import plotly.express as px
df = data_bk["New_Player_Perf"].value_counts().sort_values()
fig = px.bar(df, x='New_Player_Perf',
             hover_data=['New_Player_Perf'], color='New_Player_Perf',text_auto=True,
             labels={'pop':'distrubution of New_Player_Perf'}, height=400, width=400)

fig.show()

In [16]:
# create a copy Of Raw Data
data_all=data_bk.drop(['newer_id','new_ID','ID','match_captain','Squad_Avg_GF','Squad_avg_GA','Opp_Avg_team_GF','Opp_Avg_team_GA'], axis=1)
data_all = data_all.copy()
train = data_all.sample(frac = 0.8, random_state = 200)
test = data_all.sample(frac = 0.2, random_state = 200)
data = train.copy()


In [17]:
# Create a LabelEncoder instance
encoder = LabelEncoder()

# List of columns you want to encode 'Player_Perf',
columns_to_encode = [
    'player', 'season', 'match_day', 'Match Week', 'players_team', 'opponent_team', 'Match_id',
    'position', 'main_Pos', 'match_time', 'stadium', 'players_team_formation',
    'opponent_team_formation', 'referee', 'Previous_Game', 'Opp_Previous_Game', 'Player_Perf',
    'New_Player_Perf', 'venue','opponent_team_pwr','Opp_Avg_team_rating', 'attendance'
]

# Apply label encoding to the specified columns in the data, data_all, and test DataFrames
data[columns_to_encode] = data[columns_to_encode].apply(encoder.fit_transform)
data_all[columns_to_encode] = data_all[columns_to_encode].apply(encoder.fit_transform)
test[columns_to_encode] = test[columns_to_encode].apply(encoder.fit_transform)



In [18]:
# Normalize the features in the all Data
scaler = MinMaxScaler()
columns_to_scale = data_all.drop(columns=['player','Match Week','match_date', 'season', 'match_day', 'players_team', 'opponent_team', 'Match_id',
    'position', 'main_Pos', 'match_time', 'stadium', 'players_team_formation',
    'opponent_team_formation', 'referee', 'Previous_Game', 'Opp_Previous_Game',
    'Player_Perf','New_Player_Perf', 'venue','opponent_team_pwr','Opp_Avg_team_rating', 'attendance']).columns
data_all[columns_to_scale] = scaler.fit_transform(data_all[columns_to_scale])

In [ ]:
# Normalize the features in the train Data
scaler = MinMaxScaler()
columns_to_scale = data.drop(columns=[
'player','Match Week','match_date', 'season', 'match_day', 'players_team', 'opponent_team', 'Match_id',
    'position', 'main_Pos', 'match_time', 'stadium', 'players_team_formation',
    'opponent_team_formation', 'referee', 'Previous_Game', 'Opp_Previous_Game',
    'Player_Perf', 'New_Player_Perf','venue','opponent_team_pwr','Opp_Avg_team_rating', 'attendance'

]).columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])


In [ ]:
data_corr = data_all.copy()

# Normalize the features in the train Data
scaler = MinMaxScaler()
columns_to_scale =  ['Player_Perf', 'New_Player_Perf'
            ]

data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])




# check the skewness of the data 'Player_Perf'.. normalize player_perf, new_player_perf
plt.figure(figsize=(30,10))
sns.heatmap(data_corr.drop(['player','Match Week','season','players_team','opponent_team','Match_id','position','Previous_Game','Opp_Previous_Game','total','shotot','KeyPassTotal','PassSuccessInMatch','duelAerialWon','Touches'
,'match_time','referee','opponent_team_formation','Opp_Avg_team_rating', 'New_Player_Perf','Player_Perf','main_Pos'], axis=1).corr(numeric_only=True),annot=True, cmap="BuPu")



In [19]:
correlation_matrix = data_bk.drop(['player','Match Week','season','players_team','opponent_team','Match_id','position','Previous_Game','Opp_Previous_Game','total','shotot','KeyPassTotal','PassSuccessInMatch','duelAerialWon','Touches'], axis=1).corr(numeric_only=True)

# Save the correlation matrix to a CSV file
correlation_matrix.to_csv('correlation_matrix.csv')

print("Correlation matrix saved to 'correlation_matrix.csv'")
files.download('correlation_matrix.csv')

Correlation matrix saved to 'correlation_matrix.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Get the original labels from the encoder
original_labels = encoder.classes_

decoded_labels = encoder.inverse_transform(data_all.Player_Perf)


print(decoded_labels)

In [260]:
# here split into training and test excluding  'Previous_Game','Opp_Previous_Game','players_team','opponent_team','match_time','stadium','opponent_team_formation','referee'# 'total', 'shotot',      'KeyPassTotal', 'PassSuccessInMatch', 'duelAerialWon', 'Touches',
features =  ['player','main_Pos',
             'avg_total','avg_shotot','avg_KeyPassTotal','avg_PassSuccessInMatch','avg_duelAerialWon','avg_Touches',
             'Last_Player_rating', 'Avg_player_rating', 'Squad_Last_rating','venue',
             'Squad_Avg_att_rating', 'Squad_Avg_mid_rating', 'Squad_Avg_def_rating','Squad_Avg_gk_rating',
            'opponent_team_pwr','Opp_Avg_team_rating'
            ,'Opp_Last_Team_rating', 'Opp_avg_att_rating', 'Opp_avg_mid_rating', 'Opp_avg_def_rating','Opp_avg_gk_rating',
            ]


features_nocontext =  ['player','players_team','opponent_team','position','match_time','stadium','players_team_formation','opponent_team_formation','referee',
                       'avg_total','avg_shotot','avg_KeyPassTotal','avg_PassSuccessInMatch','avg_duelAerialWon','avg_Touches', 'Last_Player_rating']

# X_train, X_test= train_test_split(data, test_size=0.2, random_state=86, stratify=data['Player_Perf'])
X_train, X_test= train_test_split(data_all, test_size=0.3, random_state=86, stratify=data_all['Player_Perf'])

y_train = X_train.Player_Perf
y_test = X_test.Player_Perf
X_train = X_train[features]
X_test = X_test[features]

X_train_no, X_test_no= train_test_split(data_all, test_size=0.3, random_state=86,  stratify=data_all['Player_Perf'])

y_train_no = X_train_no.Player_Perf
y_test_no = X_test_no.Player_Perf
X_train_no = X_train_no[features_nocontext]
X_test_no = X_test_no[features_nocontext]



X_data_all = data_all[features]
y_data_all = data_all.Player_Perf


In [ ]:
# Define models with their corresponding hyperparameter spaces for GridSearch
models = {
     "xgb": (XGBClassifier(random_state=86), {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5],
        'subsample': [0.6, 0.8, 1.0],
    }),
    "mlp": (MLPClassifier(max_iter=500), {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'alpha': [0.0001, 0.001, 0.01],
        'batch_size': [32, 64, 128],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'learning_rate': ['constant', 'adaptive']
    }),
        "hgbc": (HistGradientBoostingClassifier(random_state=86), {
        'max_iter': [100, 200, 300],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.1, 0.2]
    }),
    "catb": (CatBoostClassifier(verbose=None, random_state=86), {
        'iterations': [100, 200, 300],
        'depth': [3, 4, 5],
        'learning_rate': [0.001, 0.01, 0.004],
    })
}

results = []

# Loop through models and perform hyperparameter tuning and evaluation
for model_nm, (model, param_grid) in models.items():
    print('{0:25} : {1:5}'.format('Model name', model_nm))

    # Perform GridSearchCV with cross-validation
    grid_search = GridSearchCV(model, param_grid,cv=10, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    Training_Score = best_model.score(X_train, y_train)
    Test_Score = best_model.score(X_test, y_test)
    y_predicted = best_model.predict(X_test)

    print('Best parameters:', grid_search.best_params_)
    print('{0:25} : {1:5}'.format('F1 Training Score', Training_Score))
    print('{0:25} : {1:5}'.format('F1 Test Score', Test_Score))


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

# Separate features (X) and target variable (y)
X = data_all[features]
y = data_all['Player_Perf']

# Define models with their corresponding hyperparameters
modelss = {
    "xgb": XGBClassifier(random_state=86, n_estimators=500, max_depth=3, subsample=0.8),
    "hgbc": HistGradientBoostingClassifier(random_state=86,
                                           max_iter=100, max_depth=5, learning_rate=0.1),
    "mlp": MLPClassifier(hidden_layer_sizes=(50,), alpha=0.0001, batch_size=64,
                         activation='relu', solver='adam', learning_rate='constant',
                         max_iter=500, random_state=86),
    "catb": CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, random_state=86, verbose=None)
}

# Loop through models and perform hyperparameter tuning and evaluation
for model_nm, model in modelss.items():
    print('{0:25} : {1:5}'.format('Model name', model_nm))

    # Perform cross-validation and get predicted labels
    y_pred_cv = cross_val_predict(model, X, y, cv=10)

    # Convert cross-validated predictions back to categorical labels
    y_pred_cv_categorical = encoder.inverse_transform(y_pred_cv)

    # Convert y to categorical labels for classification report
    y_categorical = encoder.inverse_transform(y)

    # Generate classification report for cross-validated predictions
    classification_rep = classification_report(y_categorical, y_pred_cv_categorical)
    print('Classification Report for Cross-Validated Predictions:')
    print(classification_rep)


In [ ]:
# tried using an ensemble no improvement was found
X = data_all[features]
y = data_all['New_Player_Perf']

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=86,  stratify=data_all['New_Player_Perf'])

# Train XGBoost and HistGradientBoostingClassifier
xgb_model = XGBClassifier(random_state=86, n_estimators=500, max_depth=3, subsample=0.8)
hgb_model = HistGradientBoostingClassifier(random_state=86, max_iter=100, max_depth=5, learning_rate=0.1)

xgb_model.fit(X_train, y_train)
hgb_model.fit(X_train, y_train)

# Generate predictions from both models
xgb_preds = xgb_model.predict_proba(X_test)
hgb_preds = hgb_model.predict_proba(X_test)

# Prepare features for meta-model
meta_features = np.column_stack((xgb_preds[:, 1], hgb_preds[:, 1]))

# Train the meta-model (CatBoost)
meta_model = CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, random_state=86, verbose=None)
meta_model.fit(meta_features, y_test)

# Generate final predictions using the meta-model
meta_preds = meta_model.predict(meta_features)

# Print classification report
report = classification_report(y_test, meta_preds)
print(report)


In [ ]:

# Evaluate model effectiveness non contextual
modelss = {
            "xgb"  : XGBClassifier(random_state=86),
            "hgbc" : HistGradientBoostingClassifier(random_state=86),
            "mlp" : MLPClassifier(max_iter=500),
            "catb" : CatBoostClassifier(verbose=None, random_state=86)
            }

results = []

# this part of the code run multiple models and generates a score for each
for model_nm, model in modelss.items():
    print('{0:25} : {1:5}'.format('Model name', model_nm ) )


    model.fit(X_train_no, y_train_no)
    Training_Score = model.score(X_train_no, y_train_no)
    Test_Score = model.score(X_test_no, y_test_no)
    y_predicted = model.predict(X_test_no)

    print('{0:25} : {1:5}'.format('F1 Training Score',  Training_Score) )
    print('{0:25} : {1:5}'.format('F1 Test Score',  Test_Score) )

